In [5]:
import praw
import csv
import time

# Reddit connection
reddit = praw.Reddit(
    client_id='ZvOmvXHhCkKENkIqvRmkgg',
    client_secret='IZbNrLtd5ZjkgbsyhlNEjx6G_-RUnw',
    user_agent='EVSentimentAnalysisBot v1.0'
)

# Combine subreddits
subreddits = reddit.subreddit("electricvehicles+TeslaMotors+electriccars+EVs+ElectricVehiclesEU")

# Config
target_comment_count = 5000
all_comments = []
post_limit = 2000  # Total posts to scan across all subreddits

print(" Scraping comments from multiple EV subreddits...")

for post in subreddits.top(limit=post_limit):
    post.comments.replace_more(limit=0)
    for comment in post.comments:
        if len(all_comments) >= target_comment_count:
            break
        body = comment.body.replace('\n', ' ').strip()
        if len(body) > 0:
            all_comments.append([
                post.subreddit.display_name,
                post.id,
                post.title,
                comment.id,
                body
            ])

    if len(all_comments) >= target_comment_count:
        break

    time.sleep(1)  # Pause between posts to avoid being blocked

# Save to CSV
with open('ev_comments.csv', 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['subreddit', 'post_id', 'post_title', 'comment_id', 'comment_body'])
    writer.writerows(all_comments)

print(f" Scraped {len(all_comments)} comments from multiple subreddits and saved to ev_comments.csv")


 Scraping comments from multiple EV subreddits...
 Scraped 5000 comments from multiple subreddits and saved to ev_comments.csv


In [6]:
import pandas as pd
import re

# Load CSV
df = pd.read_csv('ev_comments.csv')

# Cleaning function
def clean_comment(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove non-letters
    text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
    return text

# Apply cleaning
df['clean_comment'] = df['comment_body'].astype(str).apply(clean_comment)

# Remove comments that are too short (e.g. < 5 words)
df = df[df['clean_comment'].str.split().str.len() >= 5]

# Save cleaned version
df.to_csv('ev_comments_cleaned.csv', index=False)

print(f" Cleaned and saved {len(df)} comments to ev_comments_cleaned.csv")


 Cleaned and saved 3903 comments to ev_comments_cleaned.csv


In [7]:
df_cleaned = pd.read_csv("ev_comments_cleaned.csv")
df_cleaned = df_cleaned.dropna(subset=["clean_comment"]).reset_index(drop=True)


In [8]:
df_cleaned.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3903 entries, 0 to 3902
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subreddit      3903 non-null   object
 1   post_id        3903 non-null   object
 2   post_title     3903 non-null   object
 3   comment_id     3903 non-null   object
 4   comment_body   3903 non-null   object
 5   clean_comment  3903 non-null   object
dtypes: object(6)
memory usage: 183.1+ KB
